## Load processed metadata combined csv so we can start creating the system

In [9]:
import pandas as pd

qualified_movies_df = pd.read_csv('.\\data\\processed_movies\\qualified_all_metadata.csv')

## Define methods for getting recommendations and saving the results to CSVs

In [10]:
from operator import itemgetter
import os
import pandas as pd

pd.options.mode.chained_assignment = None

dict_of_movie_id_index = dict(zip(list(qualified_movies_df['movie_id']), list(qualified_movies_df.index)))

"""Here the function is like this because we are testing, in production we will calculate just the 
movie vector with the count vectorizer matrix"""


def get_recommendations(similarity_matrix, movie_id, top_n=50):
    movie_idx = dict_of_movie_id_index[movie_id]
    sim_scores = list(enumerate(similarity_matrix[movie_idx]))
    sim_scores = sorted(sim_scores, key=itemgetter(1), reverse=True)
    sim_scores = sim_scores[0:top_n + 1]
    movie_indices = [i[0] for i in sim_scores]
    return qualified_movies_df.iloc[movie_indices], sim_scores


def save_recommendations(movie_id, df_to_save, top_n, replace=False, recommendation_type=''):
    print(df_to_save[['title']])
    movie_title = df_to_save.loc[df_to_save['movie_id'] == movie_id, 'title'].values[0]

    dir_to_save_csv = '.\\data\\recommendations\\{}_{}\\'.format(movie_id, movie_title.replace(':', ' '))
    file_name = 'top_{}_{}.csv'.format(top_n, recommendation_type)

    """Create dir if doesnt exist"""
    if not os.path.isdir(dir_to_save_csv):
        os.makedirs(dir_to_save_csv)

    if replace:
        df_to_save.to_csv(dir_to_save_csv + file_name, index=False)
        return 0
    """Check if file with this name exists"""
    if os.path.isfile(dir_to_save_csv + file_name):
        print('File {} already exist! Check wassup'.format(dir_to_save_csv + file_name))
        return 1
    else:
        df_to_save.to_csv(dir_to_save_csv + file_name, index=False)
        return 0


def remove_0_from_dict_values_and_round(curr_dict):
    return {k: round(v, 3) for k, v in curr_dict.items() if v != 0}


def get_features_for_recommendation(list_of_recommended_movie_indices_sim_scores, vectorizer, vectorizer_matrix):
    """create dict with main movie words"""
    main_movie_dict = list(map(lambda row: dict(zip(vectorizer.get_feature_names(), row)),
                               vectorizer_matrix[list_of_recommended_movie_indices_sim_scores[0][0]].toarray()))[
        0]

    """get only the movies indices"""
    list_of_rec_movies_indices = [i[0] for i in list_of_recommended_movie_indices_sim_scores[1:]]

    """create list of dicts with recommended movies words"""
    rec_movies_list_of_dicts = list(map(lambda row: dict(zip(vectorizer.get_feature_names(), row)),
                                        vectorizer_matrix[list_of_rec_movies_indices].toarray()))

    main_movie_dict = remove_0_from_dict_values_and_round(main_movie_dict)

    list_of_result = [main_movie_dict]
    for rec_dict in rec_movies_list_of_dicts:
        rec_dict = remove_0_from_dict_values_and_round(rec_dict)
        res_rec_dict = dict()
        for word, count in main_movie_dict.items():
            if word in rec_dict.keys():
                res_rec_dict[word] = rec_dict[word]
        list_of_result.append(res_rec_dict)

    return list_of_result


def format_data_before_saving_recommendations(recommendations_df, list_of_recommendation_features_dicts,
                                              recommendation_movies_indices_sim_scores):
    df = recommendations_df[
        ['movie_id', 'title', 'release_year', 'vote_count', 'vote_average', 'popularity', 'genres']]

    df['rec_features'] = list_of_recommendation_features_dicts

    for item in recommendation_movies_indices_sim_scores:
        df.loc[item[0], 'similarity_score'] = item[1]
    return df


def analyze_movies_from_directory(vectorizer, vectorizer_matrix, similarity_matrix, top_n, recommendation_type,
                                  replace=False):
    for movie_dir in os.listdir('C:\\PythonProjects\\MoviesRecommendationSystem-Production\\data\\recommendations\\'):
        movie_id = int(movie_dir.split('_')[0])

        """get recommendations"""
        recommendations_df, recommendation_movies_indices_sim_scores = get_recommendations(
            similarity_matrix=similarity_matrix,
            movie_id=movie_id,
            top_n=top_n)

        """get features by witch it was recommended"""
        list_of_recommendation_features_dicts = get_features_for_recommendation(vectorizer=vectorizer,
                                                                                vectorizer_matrix=vectorizer_matrix,
                                                                                list_of_recommended_movie_indices_sim_scores=
                                                                                recommendation_movies_indices_sim_scores)

        """get df which is going to be saved"""
        processed_rec_df = format_data_before_saving_recommendations(recommendations_df,
                                                                     list_of_recommendation_features_dicts,
                                                                     recommendation_movies_indices_sim_scores)

        """save df so i can analyze"""
        save_recommendations(movie_id, processed_rec_df, top_n, replace, recommendation_type)

        print('{} completed!'.format(movie_dir))


## Count Vectorizer method

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

"""Regex to keep keywords and not separate them, like 'hotel room', is hotel+room,
strip accents to unicode to make 'fiancé' to 'fiance' and make lowercase"""
count_vectorizer = CountVectorizer(analyzer='word', lowercase=True,
                                   stop_words='english', ngram_range=(1, 1), token_pattern=r'[a-zA-Z0-9+_]+',
                                   strip_accents='unicode')
count_vectorizer_matrix = count_vectorizer.fit_transform(qualified_movies_df['all_metadata'])

In [12]:
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

count_vect_sim_matrix_cos = cosine_similarity(count_vectorizer_matrix, count_vectorizer_matrix)
count_vect_sim_matrix_dot = linear_kernel(count_vectorizer_matrix, count_vectorizer_matrix)

print('Count Vectorizer - Cosine similarity matrix')
print(count_vect_sim_matrix_cos)
print('------------------------')
print('Count Vectorizer - Euclidean dot similarity matrix')
print(count_vect_sim_matrix_dot)

Count Vectorizer - Cosine similarity matrix
[[1.         0.05986843 0.05609927 ... 0.05296271 0.13340747 0.10567049]
 [0.05986843 1.         0.05205792 ... 0.09829464 0.06189845 0.06537205]
 [0.05609927 0.05205792 1.         ... 0.02302656 0.08700222 0.06125639]
 ...
 [0.05296271 0.09829464 0.02302656 ... 1.         0.13689642 0.14457873]
 [0.13340747 0.06189845 0.08700222 ... 0.13689642 1.         0.18208926]
 [0.10567049 0.06537205 0.06125639 ... 0.14457873 0.18208926 1.        ]]
------------------------
Count Vectorizer - Euclidean dot similarity matrix
[[31.  2.  2. ...  2.  4.  3.]
 [ 2. 36.  2. ...  4.  2.  2.]
 [ 2.  2. 41. ...  1.  3.  2.]
 ...
 [ 2.  4.  1. ... 46.  5.  5.]
 [ 4.  2.  3. ...  5. 29.  5.]
 [ 3.  2.  2. ...  5.  5. 26.]]


## TF-IDF Vectorizer method

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

"""Regex to keep keywords and not separate them, like 'hotel room', is hotel+room,
strip accents to unicode to make 'fiancé' to 'fiance' and make lowercase"""
tfidf_vectorizer = TfidfVectorizer(analyzer='word', lowercase=True,
                                   stop_words='english', ngram_range=(1, 1), token_pattern=r'[a-zA-Z0-9+_]+',
                                   strip_accents='unicode')
tfidf_vectorizer_matrix = tfidf_vectorizer.fit_transform(qualified_movies_df['all_metadata'])

In [6]:
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

tfidf_vect_sim_matrix_cos = cosine_similarity(tfidf_vectorizer_matrix, tfidf_vectorizer_matrix)
tfidf_vect_sim_matrix_dot = linear_kernel(tfidf_vectorizer_matrix, tfidf_vectorizer_matrix)

print('TF-IDF - Cosine similarity matrix')
print(tfidf_vect_sim_matrix_cos)
print('------------------------')
print('TF-IDF - Euclidean dot similarity matrix')
print(tfidf_vect_sim_matrix_dot)

TF-IDF - Cosine similarity matrix
[[1.         0.00891925 0.02053735 ... 0.00438798 0.02985328 0.0145613 ]
 [0.00891925 1.         0.01084292 ... 0.05140639 0.00872399 0.00972063]
 [0.02053735 0.01084292 1.         ... 0.0035556  0.01472711 0.01209015]
 ...
 [0.00438798 0.05140639 0.0035556  ... 1.         0.02378984 0.03337236]
 [0.02985328 0.00872399 0.01472711 ... 0.02378984 1.         0.02424314]
 [0.0145613  0.00972063 0.01209015 ... 0.03337236 0.02424314 1.        ]]
------------------------
TF-IDF - Euclidean dot similarity matrix
[[1.         0.00891925 0.02053735 ... 0.00438798 0.02985328 0.0145613 ]
 [0.00891925 1.         0.01084292 ... 0.05140639 0.00872399 0.00972063]
 [0.02053735 0.01084292 1.         ... 0.0035556  0.01472711 0.01209015]
 ...
 [0.00438798 0.05140639 0.0035556  ... 1.         0.02378984 0.03337236]
 [0.02985328 0.00872399 0.01472711 ... 0.02378984 1.         0.02424314]
 [0.0145613  0.00972063 0.01209015 ... 0.03337236 0.02424314 1.        ]]


## My own weighting method

In [7]:
qualified_movies_df.fillna('', inplace=True)

In [11]:
"""Calculate mean values of all attributes"""
count_keywords = 0
count_actors = 0
count_directors = 0
count_producers = 0
count_screenplays = 0
count_composers = 0
count_genres = 0

for index, row in qualified_movies_df.iterrows():
    count_keywords += len(row['preprocessed_keywords'].split())
    count_actors += len(row['processed_actors'].split())
    count_directors += len(row['processed_directors'].split())
    count_producers += len(row['processed_producers'].split())
    count_screenplays += len(row['processed_screenplays'].split())
    count_genres += len(row['processed_genres'].split())
    count_composers += len(row['processed_composers'].split())
    
keywords_mean = count_keywords/len(qualified_movies_df)
actors_mean = count_actors/len(qualified_movies_df)
producers_mean = count_producers/len(qualified_movies_df)
directors_mean = count_directors/len(qualified_movies_df)
screenplays_mean = count_screenplays/len(qualified_movies_df)
genres_mean = count_genres/len(qualified_movies_df)
composers_mean = count_composers/len(qualified_movies_df)

print("Keywords mean: ", keywords_mean)
print('Actors mean: ', actors_mean)
print('Producers mean: ', producers_mean)
print('Directors mean: ', directors_mean)
print('Screenplays mean: ', screenplays_mean)
print('Genres mean: ', genres_mean)
print('Composers mean: ', composers_mean)

Keywords mean:  21.880231639905237
Actors mean:  2.979029569184873
Producers mean:  1.8540844081775907
Directors mean:  1.0951127489690269
Screenplays mean:  0.9548126700008774
Genres mean:  2.497060629990348
Composers mean:  1.0715977888918136


In [16]:
"""Create weights for different categories of attributes"""

list_of_movie_dicts = []
for index, row in qualified_movies_df.iterrows():
        
    list_of_actors = row['processed_actors'].split()
    list_of_producers = row['processed_producers'].split()
    list_of_directors = row['processed_directors'].split()
    list_of_eras = row['processed_eras'].split()
    list_of_runtime = row['processed_runtime'].split()
    list_of_composers = row['processed_composers'].split()
    list_of_genres = row['processed_genres'].split()
    list_of_keywords = row['preprocessed_keywords'].split()
    list_of_screenplays = row['processed_screenplays'].split()
    
    
    num_to_substract = 1
    
    dict_of_movie_metadata = dict()
    for item in list_of_actors:
        dict_of_movie_metadata[item] = 2*actors_mean/len(list_of_actors)
        
    for item in list_of_producers:
        dict_of_movie_metadata[item] = producers_mean/len(list_of_producers)
        
    for item in list_of_directors:
        dict_of_movie_metadata[item] = 2*directors_mean/len(list_of_directors)
        
    for item in list_of_eras:
        dict_of_movie_metadata[item] = 2/len(list_of_eras)
        
    for item in list_of_runtime:
        dict_of_movie_metadata[item] = 1/len(list_of_runtime)
        
    for item in list_of_genres:
        dict_of_movie_metadata[item] = 2*genres_mean/len(list_of_genres)
        
    for item in list_of_keywords:
        dict_of_movie_metadata[item] = 3*keywords_mean/len(list_of_keywords)
        
    for item in list_of_screenplays:
        dict_of_movie_metadata[item] = screenplays_mean/len(list_of_screenplays)
        
    for item in list_of_composers:
        dict_of_movie_metadata[item] = composers_mean/len(list_of_composers)
        
    list_of_movie_dicts.append(dict_of_movie_metadata)

In [34]:
list_of_movie_dicts[0]

{'+commit+': 5.209578961882199,
 '+country+': 5.209578961882199,
 '+crime+': 5.209578961882199,
 '+dream+': 5.209578961882199,
 '+escape+': 5.209578961882199,
 '+father+': 5.209578961882199,
 '+frame+': 5.209578961882199,
 '+helsinki+': 5.209578961882199,
 '+jail+': 5.209578961882199,
 '+leave+': 5.209578961882199,
 '+life+': 5.209578961882199,
 '+miner+': 5.209578961882199,
 '+plan+': 5.209578961882199,
 '+prison+': 5.209578961882199,
 '+start+': 5.209578961882199,
 '+thing+': 5.209578961882199,
 '+underdog+': 5.209578961882199,
 '1980_to_1990': 2.0,
 'actorMattiPellonpää': 1.9860197127899155,
 'actorSusannaHaavisto': 1.9860197127899155,
 'actorTuroPajala': 1.9860197127899155,
 'committed+suicide': 5.209578961882199,
 'directorAkiKaurismäki': 2.1902254979380538,
 'factory+worker': 5.209578961882199,
 'falling+in+love': 5.209578961882199,
 'genreCrime': 3.7455909449855223,
 'genreDrama': 3.7455909449855223,
 'independent+film': 5.209578961882199,
 'no_composer_available': 1.07159778889

In [17]:
from sklearn.feature_extraction import DictVectorizer

own_weighting_vectorizer = DictVectorizer()
own_weighting_vectorizer_matrix = own_weighting_vectorizer.fit_transform(list_of_movie_dicts)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

own_weighting_sim_matrix_cos = cosine_similarity(own_weighting_vectorizer_matrix, own_weighting_vectorizer_matrix)

print('Own Weighting - Cosine similarity matrix')
print(own_weighting_sim_matrix_cos)

Own Weighting - Cosine similarity matrix
[[1.         0.03064765 0.05329825 ... 0.01030581 0.07091    0.02341786]
 [0.03064765 1.         0.0395682  ... 0.08245494 0.01954212 0.01957936]
 [0.05329825 0.0395682  1.         ... 0.0456733  0.03529403 0.02357419]
 ...
 [0.01030581 0.08245494 0.0456733  ... 1.         0.09708869 0.09727369]
 [0.07091    0.01954212 0.03529403 ... 0.09708869 1.         0.03884534]
 [0.02341786 0.01957936 0.02357419 ... 0.09727369 0.03884534 1.        ]]


## Test different methods

In [9]:
analyze_movies_from_directory(vectorizer=count_vectorizer, vectorizer_matrix=count_vectorizer_matrix,
                              similarity_matrix=count_vect_sim_matrix_cos, top_n=10, recommendation_type='count_vect_cos', replace=True)
analyze_movies_from_directory(vectorizer=count_vectorizer, vectorizer_matrix=count_vectorizer_matrix,
                              similarity_matrix=count_vect_sim_matrix_dot, top_n=10, recommendation_type='count_vect_dot', replace=True)
analyze_movies_from_directory(vectorizer=tfidf_vectorizer, vectorizer_matrix=tfidf_vectorizer_matrix,
                              similarity_matrix=tfidf_vect_sim_matrix_dot, top_n=10, recommendation_type='tfidf_normalized', replace=True)


                                              title
3                                         Star Wars
990                         The Empire Strikes Back
9165                   Star Wars: The Force Awakens
992       Star Wars: Episode I - The Phantom Menace
994    Star Wars: Episode III - Revenge of the Sith
991                              Return of the Jedi
993    Star Wars: Episode II - Attack of the Clones
8396                  The Star Wars Holiday Special
586                                           Krull
10754                  Rogue One: A Star Wars Story
1010                                       Superman
11_Star Wars completed!


                                                   title
76     The Lord of the Rings: The Fellowship of the Ring
77                 The Lord of the Rings: The Two Towers
78         The Lord of the Rings: The Return of the King
8009                 The Hobbit: The Desolation of Smaug
7784                   The Hobbit: An Unexpected Journey
9006           The Hobbit: The Battle of the Five Armies
79                                 The Lord of the Rings
10315                                     Doctor Strange
5119                                          Snow Queen
4500                                              Exiled
1891                                    The Lovely Bones
120_The Lord of the Rings  The Fellowship of the Ring completed!


                                                   title
79                                 The Lord of the Rings
77                 The Lord of the Rings: The Two Towers
8009                 The Hobbit: The Desolation of Smaug
76     The Lord of the Rings: The Fellowship of the Ring
9006           The Hobbit: The Battle of the Five Armies
7784                   The Hobbit: An Unexpected Journey
2269                                        Fitzcarraldo
5737                                 Time of the Gypsies
10298        Miss Peregrine's Home for Peculiar Children
112                        Star Trek IV: The Voyage Home
80                                     Princess Mononoke
123_The Lord of the Rings completed!


                       title
4226               Eagle Eye
835    Live Free or Die Hard
1851                 Shooter
4600                 Ghajini
124          Minority Report
10403       Now You See Me 2
1076                 Hostage
43                Miami Vice
2600                The Rock
4184        Cold Creek Manor
8416          Now You See Me
13027_Eagle Eye completed!


                                title
5                        Forrest Gump
6256         The Mirror Has Two Faces
849       What's Eating Gilbert Grape
3003                 Corrina, Corrina
10386                Miss You Already
4008              Defending Your Life
10755                      The Choice
7598                    Twelfth Night
9815   Goliyon Ki Raasleela Ram-Leela
2320               Great Expectations
2598                     Philadelphia
13_Forrest Gump completed!


...

                           title
1983                    Superbad
6926        Get Him to the Greek
5399         Sex, Party and Lies
2635   Forgetting Sarah Marshall
1790      The 40 Year Old Virgin
1572                  Knocked Up
2843           Pineapple Express
6073                    Kick-Ass
10198                 Trainwreck
5734             Bart Got a Room
5210               Adventureland
8363_Superbad completed!


                                title
8608                      On the Road
891            The Motorcycle Diaries
4968                     American Gun
9481                  Tom at the Farm
1953                     Summer Hours
9938             Clouds of Sils Maria
10883                Personal Shopper
3533      The World According to Garp
9949                  The Dark Valley
2444                           Convoy
10844  It's Only the End of the World
83770_On the Road completed!


                                    title
662                   Mission: Impossible
663                Mission: Impossible II
664               Mission: Impossible III
9435   Mission: Impossible - Rogue Nation
10463                  Dying of the Light
441                                Dr. No
3781                            Get Smart
413                      The Last Samurai
460                   You Only Live Twice
998                           Vanilla Sky
3316                   Jumpin' Jack Flash
954_Mission  Impossible completed!


                                      title
2597               The Fast and the Furious
8598                       Fast & Furious 6
4498                         Fast & Furious
384                        2 Fast 2 Furious
9359                              Furious 7
7878                              Fast Five
2749                                Stealth
2206                               S.W.A.T.
1553                              Dark Blue
2471  The Fast and the Furious: Tokyo Drift
1843                                    xXx
9799_The Fast and the Furious completed!


                                title
55                          Gladiator
5588                           Attila
6904                        Spartacus
2532                  The Last Legion
672                         Spartacus
2300              Conan the Barbarian
2083               Asterix vs. Caesar
3847                          Amistad
7792                        The Eagle
6395                            Agora
2432  Asterix & Obelix Take on Caesar
98_Gladiator completed!


In [19]:
analyze_movies_from_directory(vectorizer=own_weighting_vectorizer, vectorizer_matrix=own_weighting_vectorizer_matrix,
                              similarity_matrix=own_weighting_sim_matrix_cos, top_n=10, recommendation_type='own_weighting_cos', replace=True)

                                              title
3                                         Star Wars
990                         The Empire Strikes Back
9165                   Star Wars: The Force Awakens
994    Star Wars: Episode III - Revenge of the Sith
992       Star Wars: Episode I - The Phantom Menace
991                              Return of the Jedi
993    Star Wars: Episode II - Attack of the Clones
8396                  The Star Wars Holiday Special
10754                  Rogue One: A Star Wars Story
112                   Star Trek IV: The Voyage Home
586                                           Krull
11_Star Wars completed!


                                                   title
76     The Lord of the Rings: The Fellowship of the Ring
77                 The Lord of the Rings: The Two Towers
8009                 The Hobbit: The Desolation of Smaug
78         The Lord of the Rings: The Return of the King
9006           The Hobbit: The Battle of the Five Armies
7784                   The Hobbit: An Unexpected Journey
79                                 The Lord of the Rings
752                                           The Hobbit
8256                                            Warcraft
3681                                           Excalibur
10565                                       Last Knights
120_The Lord of the Rings  The Fellowship of the Ring completed!


                                                   title
79                                 The Lord of the Rings
8009                 The Hobbit: The Desolation of Smaug
77                 The Lord of the Rings: The Two Towers
76     The Lord of the Rings: The Fellowship of the Ring
7784                   The Hobbit: An Unexpected Journey
9006           The Hobbit: The Battle of the Five Armies
10902                            Night Of The Living Deb
752                                           The Hobbit
2722                                      Brother Bear 2
1902                                    Punch-Drunk Love
10565                                       Last Knights
123_The Lord of the Rings completed!


                                         title
4226                                 Eagle Eye
835                      Live Free or Die Hard
2600                                  The Rock
10099            The Yellow Eyes of Crocodiles
9064                           Stories We Tell
4184                          Cold Creek Manor
2396                               The Matador
6584                                     Chloe
2744   Miss Congeniality 2: Armed and Fabulous
9945                                 Mea Culpa
5067                                 Interiors
13027_Eagle Eye completed!


                             title
5                     Forrest Gump
10755                   The Choice
2598                  Philadelphia
849    What's Eating Gilbert Grape
8541                          W.E.
8460                    True Women
3357                   Wonder Boys
2273             Universal Soldier
3488               Cinema Paradiso
7248                   The Cyclone
3904            Bullet in the Head
13_Forrest Gump completed!


...

                               title
1983                        Superbad
5399             Sex, Party and Lies
8005          Tu la conosci Claudia?
7941                      La matassa
4794                           Boudu
7069                 Cado dalle nubi
7937                     Wild Cherry
5210                   Adventureland
6428                      Glory Daze
4163                    Still Smokin
9931  The Three Brothers: The Return
8363_Superbad completed!


                        title
8608              On the Road
10570       Mississippi Grind
4968             American Gun
10879          American Honey
223                 Snow Cake
10442               Safelight
6054   Werckmeister Harmonies
10398        The Sea of Trees
687               The Hustler
8794       Dumb and Dumber To
11214           The Last Word
83770_On the Road completed!


                                    title
662                   Mission: Impossible
663                Mission: Impossible II
664               Mission: Impossible III
1567                            Drop Zone
3781                            Get Smart
10463                  Dying of the Light
9435   Mission: Impossible - Rogue Nation
3292            The Man with One Red Shoe
9191                                RED 2
441                                Dr. No
5558                          Crime Story
954_Mission  Impossible completed!


                                      title
2597               The Fast and the Furious
384                        2 Fast 2 Furious
8598                       Fast & Furious 6
4498                         Fast & Furious
1553                              Dark Blue
7878                              Fast Five
2667                             Waist Deep
8867                              Hit & Run
2471  The Fast and the Furious: Tokyo Drift
9831                                    Spy
5880                             Stone Cold
9799_The Fast and the Furious completed!


                                              title
55                                        Gladiator
5588                                         Attila
6904                                      Spartacus
9477                                       Hercules
1580                              American Gangster
994    Star Wars: Episode III - Revenge of the Sith
6054                         Werckmeister Harmonies
8304                                          Arena
3969                                   Body of Lies
4647  The Librarian: Return to King Solomon's Mines
842                                      Apocalypto
98_Gladiator completed!


## Save the chosen method

In [13]:
import pickle

with open('.\\serialized_objects\\movie_based\\count_vectorizer.pkl', 'wb') as f:
    pickle.dump(count_vectorizer, f)
    
with open('.\\serialized_objects\\movie_based\\count_vectorizer_matrix.pkl', 'wb') as f:
    pickle.dump(count_vectorizer_matrix, f)

## Clean directory from useless CSV files if needed

In [8]:
"""Clean directory"""
import os
import glob

files = glob.glob('.\\data\\recommendations\\**\\*.csv', recursive=True)

for f in files:
    try:
        os.remove(f)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))